In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

StatementMeta(, 2acee419-7772-4f29-8810-dbb77bf45afa, 3, Finished, Available)

In [2]:
df_sales= pd.read_excel("abfss://bb3ae42b-617a-4d5a-baf9-d45cf59b3f04@onelake.dfs.fabric.microsoft.com/ff98c3a6-7203-438e-b395-a08c628364e2/Files/Current/Sales*.xlsx",sheet_name="Sales")

StatementMeta(, 2acee419-7772-4f29-8810-dbb77bf45afa, 4, Finished, Available)

In [3]:
df1=spark.createDataFrame(df_sales)

StatementMeta(, 2acee419-7772-4f29-8810-dbb77bf45afa, 5, Finished, Available)

In [4]:
display(df1.head(10))

StatementMeta(, 2acee419-7772-4f29-8810-dbb77bf45afa, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, a918db5b-c163-4bb6-8265-f786f7d954b5)

In [5]:
df_Returns= pd.read_excel("abfss://bb3ae42b-617a-4d5a-baf9-d45cf59b3f04@onelake.dfs.fabric.microsoft.com/ff98c3a6-7203-438e-b395-a08c628364e2/Files/Current/Sales*.xlsx",sheet_name="Returns")

StatementMeta(, 2acee419-7772-4f29-8810-dbb77bf45afa, 7, Finished, Available)

In [6]:
df2=spark.createDataFrame(df_Returns)

StatementMeta(, 2acee419-7772-4f29-8810-dbb77bf45afa, 8, Finished, Available)

In [7]:
display(df2.head(10))

StatementMeta(, 2acee419-7772-4f29-8810-dbb77bf45afa, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, abd74e91-9368-4987-8c9d-026e20288bea)

In [25]:
df_final=df1.join(df2,df1.Order_ID==df2.Order_ID,how="left").drop(df2.Order_ID,df2.Customer_Name,df2.Sales_Amount)

StatementMeta(, 2acee419-7772-4f29-8810-dbb77bf45afa, 27, Finished, Available)

In [26]:
df_mod=df_final.withColumns({"Order_Year": year("Order_Date"), \
                "Order_Month": month("Order_Date"), \
                "Created_TS": current_timestamp(),\
                "Modified_TS": current_timestamp(),\
                })

StatementMeta(, 2acee419-7772-4f29-8810-dbb77bf45afa, 28, Finished, Available)

In [27]:
df_mod.createOrReplaceTempView("ViewSales")

StatementMeta(, 2acee419-7772-4f29-8810-dbb77bf45afa, 29, Finished, Available)

In [28]:
%%sql
create table if not exists Project_Lakehouse.Bronze_Sales
(
    Order_ID	string,
    Order_Date	Date,
    Shipping_Date	date,
    Aging	int,
    Ship_Mode	string,
    Product_Category	string,
    Product	 string,
    Sales 	float,
    Quantity	float,
    Discount	 float,
    Profit 	 float,
    Shipping_Cost 	float,
    Order_Priority	string,
    Customer_ID	string,
    Customer_Name	string,
    Segment	string,
    City	string,
    State	string,
    Country	string,
    Region string,
	Return string,
    Order_Year int,
    Order_Month int,
    Created_TS  TIMESTAMP,
    Modified_TS  TIMESTAMP
)
using DELTA
PARTITIONED by(Order_Year,Order_Month)



StatementMeta(, 2acee419-7772-4f29-8810-dbb77bf45afa, 30, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [29]:
%%sql
Merge into  Project_Lakehouse.Bronze_Sales as BS
using ViewSales as VS
on  BS.Order_Year=VS.Order_Year and BS.Order_Month=VS.Order_Month and BS.Order_ID=VS.Order_ID
when matched then 
update SET
BS.Order_Date	=	VS.Order_Date	,
BS.Shipping_Date	=	VS.Shipping_Date	,
BS.Aging	=	VS.Aging	,
BS.Ship_Mode	=	VS.Ship_Mode	,
BS.Product_Category	=	VS.Product_Category	,
BS.Product	=	VS.Product	,
BS.Sales 	=	VS.Sales 	,
BS.Quantity	=	VS.Quantity	,
BS.Discount	=	VS.Discount	,
BS.Profit 	=	VS.Profit 	,
BS.Shipping_Cost 	=	VS.Shipping_Cost 	,
BS.Order_Priority	=	VS.Order_Priority	,
BS.Customer_ID	=	VS.Customer_ID	,
BS.Customer_Name	=	VS.Customer_Name	,
BS.Segment	=	VS.Segment	,
BS.City	=	VS.City	,
BS.State	=	VS.State	,
BS.Country	=	VS.Country	,
BS.Region  	=	VS.Region  	,
BS.Return  	=	VS.Return  	,
BS.Modified_TS	=	VS.Modified_TS	

when not matched then 
INSERT
(
BS.Order_ID,    
BS.Order_Date	,
BS.Shipping_Date	,
BS.Aging	,
BS.Ship_Mode	,
BS.Product_Category	,
BS.Product	,
BS.Sales 	,
BS.Quantity	,
BS.Discount	,
BS.Profit 	,
BS.Shipping_Cost 	,
BS.Order_Priority	,
BS.Customer_ID	,
BS.Customer_Name	,
BS.Segment	,
BS.City	,
BS.State	,
BS.Country	,
BS.Region  	,
BS.Return,
BS.Order_Year  	,
BS.Order_Month  	,
BS.Created_TS   	,
BS.Modified_TS	
)
values
(
VS.Order_ID,
VS.Order_Date	,
VS.Shipping_Date	,
VS.Aging	,
VS.Ship_Mode	,
VS.Product_Category	,
VS.Product	,
VS.Sales 	,
VS.Quantity	,
VS.Discount	,
VS.Profit 	,
VS.Shipping_Cost 	,
VS.Order_Priority	,
VS.Customer_ID	,
VS.Customer_Name	,
VS.Segment	,
VS.City	,
VS.State	,
VS.Country	,
VS.Region  	,
VS.Return,
VS.Order_Year  	,
VS.Order_Month  	,
VS.Created_TS   	,
VS.Modified_TS	

)


StatementMeta(, 2acee419-7772-4f29-8810-dbb77bf45afa, 31, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>